In [3]:
import pandas as pd
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_percentage_error


2024-11-10 20:03:23.089106: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [95]:

def ann_system(hub1_name, hub2_name, validation_size, test_size, window_size, mode="test", lr = 0.003):

    hub1 = pd.read_csv(f"../../data/interpolated/{hub1_name}_close_interpolated.csv")
    hub2 = pd.read_csv(f"../../data/interpolated/{hub2_name}_close_interpolated.csv")

    hub1 = hub1.rename(columns={"CLOSE": "hub1_CLOSE"})
    hub2 = hub2.rename(columns={"CLOSE": "hub2_CLOSE"})
    hub1_hub2_diff = pd.DataFrame(hub1["hub1_CLOSE"] - hub2["hub2_CLOSE"], columns=["hub1_hub2_diff"], index=hub1.index)

    # Shift columns and store in new columns for hub1, hub2, and hub1_hub2_diff
    for i in range(window_size, window_size + 6):
        hub1[f"hub1_CLOSE-{i- window_size}"] = hub1["hub1_CLOSE"].shift(i)
        hub2[f"hub2_CLOSE-{i - window_size}"] = hub2["hub2_CLOSE"].shift(i)
        hub1_hub2_diff[f"hub1_hub2_diff-{i - window_size}"] = hub1_hub2_diff["hub1_hub2_diff"].shift(i)

    # Concatenate and drop NaN rows in one step
    data = pd.concat([hub1, hub2, hub1_hub2_diff], axis=1).dropna()

    features = [
        'hub1_CLOSE-0', #'hub1_CLOSE-1', #'hub1_CLOSE-2', 'hub1_CLOSE-3', 'hub1_CLOSE-4', 'hub1_CLOSE-5',
        'hub2_CLOSE-0', #'hub2_CLOSE-1', #'hub2_CLOSE-2', 'hub2_CLOSE-3', 'hub2_CLOSE-4', 'hub2_CLOSE-5',
        'hub1_hub2_diff-0', 'hub1_hub2_diff-1', 'hub1_hub2_diff-2', 'hub1_hub2_diff-3', 'hub1_hub2_diff-4', 'hub1_hub2_diff-5',# 'hub1_hub2_diff-6'
    ]

    X = data[features].values
    y = data[['hub1_CLOSE', 'hub2_CLOSE']].values

    if mode == "validation":
        X_train, X_test = X[:-test_size], X[-test_size:]
        y_train, y_test = y[:-test_size], y[-test_size:]

        X_train, X_val = X_train[:-validation_size], X_train[-validation_size:]
        y_train, y_val = y_train[:-validation_size], y_train[-validation_size:]

        print(X_train.shape, X_val.shape, X_test.shape)

    else:
        X_train, X_test = X[:-test_size], X[-test_size:]
        y_train, y_test = y[:-test_size], y[-test_size:]

        print(X_train.shape, X_test.shape)

    
    keras.utils.set_random_seed(42)
    # Build a simple ANN model
    model = Sequential([
        Dense(2, activation='relu'),
        #Dense(1, activation='linear'),
        Dense(2)  # 2 outputs for both hubs
    ])
    #lr = 0.0033595696621040246 TTF-THE
    # Compile the model
    
    model.compile(optimizer=Adam(learning_rate=lr), loss='mape')

    # Train the model
    history = model.fit(X_train, y_train, epochs=25, batch_size=1,
                        shuffle=False,
                        verbose=1)

    # Evaluate the model

    if mode == "validation":
        test_loss = model.evaluate(X_val, y_val, verbose=1)
        print(f"Val Loss: {test_loss}")

        # Predict on test data
        predictions = model.predict(X_val)

        # Calculate MAPE
        mape_hub1 = mean_absolute_percentage_error(y_val[:, 0], predictions[:, 0]) * 100
        print(f"MAPE for {hub1_name}: {mape_hub1:.2f}%")

        # Calculate MAPE for Hub 2
        mape_hub2 = mean_absolute_percentage_error(y_val[:, 1], predictions[:, 1]) * 100
        print(f"MAPE for {hub2_name}: {mape_hub2:.2f}%")
    else:
        test_loss = model.evaluate(X_test, y_test, verbose=1)
        print(f"Test Loss: {test_loss}")

        # Predict on test data
        predictions = model.predict(X_test)

        # Calculate MAPE
        mape_hub1 = mean_absolute_percentage_error(y_test[:, 0], predictions[:, 0]) * 100
        print(f"MAPE for {hub1_name}: {mape_hub1:.2f}%")

        # Calculate MAPE for Hub 2
        mape_hub2 = mean_absolute_percentage_error(y_test[:, 1], predictions[:, 1]) * 100
        print(f"MAPE for {hub2_name}: {mape_hub2:.2f}%")

    return hub1, hub2

In [98]:


hub1_name = "ttf"
hub2_name = "nbp"
validation_size = 250
test_size = 250
window_size = 5

#=================old=====================
#lr = 0.0033595696621040246 # TTF-THE
#lr = 0.0020391667368093716 # TTF-NBP
#lr = 0.0022680764797231650 # THE-NBP

#=================new====================
#lr = 0.0034989953431570587 # TTF-THE
lr = 0.002011660009751183 # TTF-NBP
#lr = 0.0020055959168877725 # THE-NBP

hub1, hub2 = ann_system(hub1_name, hub2_name, validation_size, test_size, window_size, mode = "validation", lr = lr)

(1035, 8) (250, 8) (250, 8)
Epoch 1/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 79.0164
Epoch 2/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 11.4459
Epoch 3/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 8.6550
Epoch 4/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 8.4779
Epoch 5/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 8.5567
Epoch 6/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 8.5174
Epoch 7/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 8.4786
Epoch 8/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 8.5785
Epoch 9/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 8.6180
Epoch 10/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 8.5550
Epoch 11/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 8.5943
Epoch 12/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 8.5906
Epoch 13/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 8.6911
Epoch 14/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 8.6292

In [99]:
hub1, hub2 = ann_system(hub1_name, hub2_name, validation_size, test_size, window_size, mode = "test", lr = lr)

(1285, 8) (250, 8)
Epoch 1/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 73.6534
Epoch 2/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 7.9325
Epoch 3/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 7.8927
Epoch 4/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 7.8417
Epoch 5/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 7.8762
Epoch 6/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 7.8336
Epoch 7/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 7.8098
Epoch 8/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 7.7948
Epoch 9/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 7.8003
Epoch 10/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 7.8495
Epoch 11/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 7.8630
Epoch 12/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 7.8674
Epoch 13/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 7.8869
Epoch 14/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 7.8830
Epoch 15/

In [89]:
import optuna
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_percentage_error
import pandas as pd

# Modify the ann_system function to accept the learning rate as an argument
def optuna_ann_system(hub1_name, hub2_name, validation_size, test_size, window_size, lr,mode="validation", verbose=True, save=True):
    hub1 = pd.read_csv(f"../../data/interpolated/{hub1_name}_close_interpolated.csv")
    hub2 = pd.read_csv(f"../../data/interpolated/{hub2_name}_close_interpolated.csv")

    hub1 = hub1.rename(columns={"CLOSE": "hub1_CLOSE"})
    hub2 = hub2.rename(columns={"CLOSE": "hub2_CLOSE"})
    hub1_hub2_diff = pd.DataFrame(hub1["hub1_CLOSE"] - hub2["hub2_CLOSE"], columns=["hub1_hub2_diff"], index=hub1.index)

    for i in range(window_size, window_size + 6):
        hub1[f"hub1_CLOSE-{i- window_size}"] = hub1["hub1_CLOSE"].shift(i)
        hub2[f"hub2_CLOSE-{i - window_size}"] = hub2["hub2_CLOSE"].shift(i)
        hub1_hub2_diff[f"hub1_hub2_diff-{i - window_size}"] = hub1_hub2_diff["hub1_hub2_diff"].shift(i)

    data = pd.concat([hub1, hub2, hub1_hub2_diff], axis=1).dropna()

    features = [
        'hub1_CLOSE-0',
        'hub2_CLOSE-0',
        'hub1_hub2_diff-0', 'hub1_hub2_diff-1', 'hub1_hub2_diff-2',
        'hub1_hub2_diff-3', 'hub1_hub2_diff-4', 'hub1_hub2_diff-5',
    ]

    X = data[features].values
    y = data[['hub1_CLOSE', 'hub2_CLOSE']].values

    X_train, X_test = X[:-test_size], X[-test_size:]
    y_train, y_test = y[:-test_size], y[-test_size:]

    X_train, X_val = X_train[:-validation_size], X_train[-validation_size:]
    y_train, y_val = y_train[:-validation_size], y_train[-validation_size:]

    keras.utils.set_random_seed(42)

    model = Sequential([
        Dense(2, activation='relu'),
        Dense(2)
    ])

    
    model.compile(optimizer=Adam(learning_rate=lr), loss='mape')
    model.fit(X_train, y_train, epochs=25, batch_size=1, shuffle=False, verbose=0)

    val_loss = model.evaluate(X_val, y_val, verbose=0)
    return val_loss


hub1_name = "the"
hub2_name = "nbp"

def objective(trial):
    lr = trial.suggest_float('lr', 2.0e-3, 3.5e-3)

    validation_loss = optuna_ann_system(hub1_name=hub1_name, hub2_name=hub2_name, validation_size=250, test_size=250, window_size=5, lr=lr)
    return validation_loss

# Run Optuna optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

# Print the best learning rate found
print("Best learning rate:", study.best_params['lr'])


[I 2024-11-10 23:10:19,636] A new study created in memory with name: no-name-4f540931-3a5a-460e-a82d-32bf5a70346e
[I 2024-11-10 23:10:35,079] Trial 0 finished with value: 10.740342140197754 and parameters: {'lr': 0.002296770340205521}. Best is trial 0 with value: 10.740342140197754.
[I 2024-11-10 23:10:52,748] Trial 1 finished with value: 10.971541404724121 and parameters: {'lr': 0.002602940817227042}. Best is trial 0 with value: 10.740342140197754.
[I 2024-11-10 23:11:08,696] Trial 2 finished with value: 11.138235092163086 and parameters: {'lr': 0.002422606379378012}. Best is trial 0 with value: 10.740342140197754.
[I 2024-11-10 23:11:24,836] Trial 3 finished with value: 10.743795394897461 and parameters: {'lr': 0.0023390239589072925}. Best is trial 0 with value: 10.740342140197754.
[I 2024-11-10 23:11:41,021] Trial 4 finished with value: 11.064925193786621 and parameters: {'lr': 0.002545308157076079}. Best is trial 0 with value: 10.740342140197754.
[I 2024-11-10 23:11:57,186] Trial 5

Best learning rate: 0.0020055959168877725


In [90]:
hub1, hub2 = ann_system(hub1_name, hub2_name, 250, 250, 5, mode = "test", lr=study.best_params['lr'])

(1285, 8) (250, 8)
Epoch 1/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 581us/step - loss: 10.4395
Epoch 2/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 549us/step - loss: 7.3005
Epoch 3/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 584us/step - loss: 7.2642
Epoch 4/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 563us/step - loss: 7.2221
Epoch 5/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 545us/step - loss: 7.1874
Epoch 6/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 573us/step - loss: 7.1625
Epoch 7/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 643us/step - loss: 7.1675
Epoch 8/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 565us/step - loss: 7.1987
Epoch 9/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 567us/step - loss: 7.1700
Epoch 10/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 565us/step - loss: 7.1903
Epoch 11/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 567us/step - loss: 7.1698
Epoch 12/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 571us/step - loss: 7.2013
Epoch 13/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 581us/step - loss: 7.1968
Epoch 14/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 560us/s